In [ ]:
### WARNING: NOTEBOOK IS UNFINISHED ###

In [1]:
####################################################################################################
# The following notebook is an isolated version of the main notebooks for Design Project 2 in the  #
# spring semester course of AE 353. The following takes code from the ZagiDemo-Template.ipynb and  #
# the DeriveEOM-Template.ipynb notebooks to create a notebook that unequivocally demonstrates the  #
# process of deriving the equations of motion for a zagi-like UAV.                                 #
####################################################################################################
# The process of determining the equations of motion follows methods learned during lecture in     #
# addition to notes from Georgia State Univeristy                                                  #
# https://scholarworks.gsu.edu/cgi/viewcontent.cgi?article=1045&context=math_theses                #
# and other resources to understand the process behind python math library commands                #
# https://en.wikipedia.org/wiki/Nelder%E2%80%93Mead_method                                         #
####################################################################################################

# Imports from ZagiDemo-Template.ipynb
import time
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display
import asyncio
import ae353_zagi

# Imports from DeriveEOM-Template.ipynb
import numpy as np
import sympy as sym
from scipy import linalg
import matplotlib.pyplot as plt
from IPython.display import display, Latex, Markdown
# from control import lqr, ctrb
from scipy.optimize import minimize
from scipy.linalg import solve_continuous_are

In [2]:
####################################################################################################
### VARIABLES AND PARAMETERS TAKEN FROM DeriveEOM-Template.ipynb
####################################################################################################

# Time
t = sym.Symbol('t', real=True)

# Components of position (meters)
p_x, p_y, p_z = sym.symbols('p_x, p_y, p_z', real=True)

# Yaw, pitch, and roll angles (radians)
psi, theta, phi = sym.symbols('psi, theta, phi', real=True)

# Components of linear velocity in the body frame (meters / second)
v_x, v_y, v_z = sym.symbols('v_x, v_y, v_z', real=True)

# Components of angular velocity in the body frame (radians / second)
w_x, w_y, w_z = sym.symbols('w_x, w_y, w_z', real=True)

# Elevon angles
delta_r, delta_l = sym.symbols('delta_r, delta_l', real=True)

#
# PARAMETERS
#

# Aerodynamic parameters
rho, S, c, b = sym.symbols('rho, S, c, b', real=True)
C_L_0, C_L_alpha, C_L_q, C_L_delta_e = sym.symbols('C_L_0, C_L_alpha, C_L_q, C_L_delta_e', real=True)
C_D_0, C_D_alpha, C_D_q, C_D_delta_e = sym.symbols('C_D_0, C_D_alpha, C_D_q, C_D_delta_e', real=True)
C_m_0, C_m_alpha, C_m_q, C_m_delta_e = sym.symbols('C_m_0, C_m_alpha, C_m_q, C_m_delta_e', real=True)
C_Y_0, C_Y_beta, C_Y_p, C_Y_r, C_Y_delta_a = sym.symbols('C_Y_0, C_Y_beta, C_Y_p, C_Y_r, C_Y_delta_a', real=True)
C_l_0, C_l_beta, C_l_p, C_l_r, C_l_delta_a = sym.symbols('C_l_0, C_l_beta, C_l_p, C_l_r, C_l_delta_a', real=True)
C_n_0, C_n_beta, C_n_p, C_n_r, C_n_delta_a = sym.symbols('C_n_0, C_n_beta, C_n_p, C_n_r, C_n_delta_a', real=True)
e, alpha_0, C_D_p, M = sym.symbols('e, alpha_0, C_D_p, M', real=True)
k, k_e = sym.symbols('k, k_e', real=True)

# Mass and inertia parameters
J_x, J_y, J_z, J_xz = sym.symbols('J_x, J_y, J_z, J_xz', real=True)
m, g = sym.symbols('m, g', real=True)

params = {
    g: 9.81,               # Gravity (m/s²)
    m: 1.56,               # Mass of the UAV (kg)
    J_x: 0.1147,           # Moment of inertia about x-axis (kg·m²)
    J_y: 0.0576,           # Moment of inertia about y-axis (kg·m²)
    J_z: 0.1712,           # Moment of inertia about z-axis (kg·m²)
    J_xz: 0.0015,          # Product of inertia (kg·m²)

    S: 0.4696,             # Wing area (m²)
    b: 1.4224,             # Wingspan (m)
    c: 0.3302,             # Mean aerodynamic chord (m)

    rho: 1.2682,           # Air density (kg/m³)

    # Lift Coefficients
    C_L_0: 0.2,            # Lift coefficient at zero AoA
    C_L_alpha: 4.8,        # Lift curve slope (1/rad)
    C_L_q: 2.2,            # Pitch rate effect on lift (1/rad)

    # Drag Coefficients
    C_D_0: 0.02,           # Zero-lift drag coefficient
    C_D_alpha: 0.30,       # Drag change per AoA (1/rad)
    C_D_q: 0.0,            # Pitch rate effect on drag (1/rad)
    C_D_p: 0.03,           # Parasitic drag coefficient

    # Pitching Moment Coefficients
    C_m_0: -0.02,          # Pitching moment at zero AoA
    C_m_alpha: -0.6,       # Pitching moment change per AoA (1/rad)
    C_m_q: -1.8,           # Pitch rate effect on moment (1/rad)
    C_m_delta_e: -0.35,    # Effect of elevator deflection on pitching moment (1/rad)

    # Side Force Coefficients
    C_Y_0: 0.0,            # Side force at zero sideslip
    C_Y_beta: -0.08,       # Side force per sideslip angle (1/rad)
    C_Y_p: 0.0,            # Side force due to roll rate
    C_Y_r: 0.0,            # Side force due to yaw rate
    C_Y_delta_a: 0.0,      # Side force due to aileron deflection

    # Roll Moment Coefficients
    C_l_0: 0.0,            # Roll moment at zero sideslip
    C_l_beta: -0.10,       # Roll moment due to sideslip (1/rad)
    C_l_p: -0.45,          # Roll damping derivative (1/rad)
    C_l_r: 0.03,           # Roll moment due to yaw rate (1/rad)
    C_l_delta_a: 0.18,     # Aileron effect on roll (1/rad)

    # Yaw Moment Coefficients
    C_n_0: 0.0,            # Yaw moment at zero sideslip
    C_n_beta: 0.008,       # Yaw moment due to sideslip (1/rad)
    C_n_p: -0.022,         # Yaw moment due to roll rate (1/rad)
    C_n_r: -0.009,         # Yaw damping derivative (1/rad)
    C_n_delta_a: -0.004,   # Aileron effect on yaw (1/rad)

    # Control Derivatives
    C_L_delta_e: 0.30,     # Effect of elevator deflection on lift (1/rad)
    C_D_delta_e: 0.32,     # Effect of elevator deflection on drag (1/rad)

    # Efficiency Factors
    e: 0.85,               # Oswald efficiency factor
    alpha_0: 0.45,         # Zero-lift angle of attack (rad)

    # Additional Drag & Lift Coefficients
    M: 50.0,               # Sigmoid blending function parameter
    k_e: 0.01,             # Drag due to elevator deflection (empirical coefficient)
    k: 0.048               # Induced drag factor
}

# Get airspeed, angle of attack, and angle of sideslip
V_a = sym.sqrt(v_x**2 + v_y**2 + v_z**2)
alpha = sym.atan(v_z / v_x)
beta = sym.asin(v_y / V_a)

# Convert from right and left elevon deflections to equivalent elevator and aileron deflections
delta_e = (delta_r + delta_l) / 2
delta_a = (-delta_r + delta_l) / 2

# Longitudinal aerodynamics
C_L = C_L_0 + C_L_alpha * alpha
F_lift = rho * V_a**2 * S * (C_L + C_L_q * (c / (2 * V_a)) * w_y + C_L_delta_e * delta_e) / 2
F_drag = rho * V_a**2 * S * ((C_D_0 + k * C_L**2) + C_D_q * (c / (2 * V_a)) * w_y + k_e * (C_L_delta_e * delta_e)**2) / 2
f_x, f_z = sym.Matrix([[sym.cos(alpha), -sym.sin(alpha)], [sym.sin(alpha), sym.cos(alpha)]]) @ sym.Matrix([[-F_drag], [-F_lift]])
tau_y = rho * V_a**2 * S * c * (C_m_0 + C_m_alpha * alpha + C_m_q * (c / (2 * V_a)) * w_y + C_m_delta_e * delta_e) / 2

# Lateral aerodynamics
f_y =   rho * V_a**2 * S *     (C_Y_0 + C_Y_beta * beta + C_Y_p * (b / (2 * V_a)) * w_x + C_Y_r * (b / (2 * V_a)) * w_z + C_Y_delta_a * delta_a) / 2
tau_x = rho * V_a**2 * S * b * (C_l_0 + C_l_beta * beta + C_l_p * (b / (2 * V_a)) * w_x + C_l_r * (b / (2 * V_a)) * w_z + C_l_delta_a * delta_a) / 2
tau_z = rho * V_a**2 * S * b * (C_n_0 + C_n_beta * beta + C_n_p * (b / (2 * V_a)) * w_x + C_n_r * (b / (2 * V_a)) * w_z + C_n_delta_a * delta_a) / 2

v_inB_ofWB = sym.Matrix([v_x, v_y, v_z])
w_inB_ofWB = sym.Matrix([w_x, w_y, w_z])

J_inB = sym.Matrix([[  J_x,    0, -J_xz],
                    [    0,  J_y,     0],
                    [-J_xz,    0,   J_z]])

Rz = sym.Matrix([[sym.cos(psi), -sym.sin(psi), 0],
                 [sym.sin(psi), sym.cos(psi), 0],
                 [0, 0, 1]])

Ry = sym.Matrix([[sym.cos(theta), 0, sym.sin(theta)],
                 [0, 1, 0],
                 [-sym.sin(theta), 0, sym.cos(theta)]])

Rx = sym.Matrix([[1, 0, 0],
                 [0, sym.cos(phi), -sym.sin(phi)],
                 [0, sym.sin(phi), sym.cos(phi)]])

R_inW_ofB = Rz @ Ry @ Rx

# First, compute the inverse of N
Ninv = sym.Matrix.hstack((Ry * Rx).T * sym.Matrix([0, 0, 1]),
                              (Rx).T * sym.Matrix([0, 1, 0]),
                                       sym.Matrix([1, 0, 0]))

# Then, take the inverse of this result to compute N
N = sym.simplify(Ninv.inv())

# Total force
f_inB = R_inW_ofB.T * sym.Matrix([0, 0, m * g]) + sym.Matrix([f_x, f_y, f_z])

# Total torque
tau_inB = sym.Matrix([tau_x, tau_y, tau_z])

f_sym = sym.Matrix.vstack(
    R_inW_ofB * v_inB_ofWB,
    N * w_inB_ofWB,
    (1 / m) * (f_inB - w_inB_ofWB.cross(m * v_inB_ofWB)),
    J_inB.inv() * (tau_inB - w_inB_ofWB.cross(J_inB * w_inB_ofWB)),
)

f = f_sym.subs(params)

In [ ]:
####################################################################################################
### THEORY ###                                                                                     #
# The following cell enacts the process defined in the theory section of the project report for    #
# DP2 written by Luke Ficalora and Joshua Veranga.                                                 #
####################################################################################################
# The following also contains annotations describing to the best of my ability the purpose of each #
# respective line of code.                                                                         #
####################################################################################################


remove = [0, 2]  
f_mod = sym.Matrix([f[i] for i in range(f.shape[0]) if i not in remove])
f_mod = sym.nsimplify(f_mod, rational=True)

f_num = sym.lambdify(
    [p_y, psi, theta, phi, v_x, v_y, v_z, w_x, w_y, w_z, delta_r, delta_l], 
    f_mod
)

f_num_for_minimize = lambda x: np.linalg.norm(f_num(*x).flatten())**2

x0 = [0, 0, 0, 0, 5, 0, -0.5, 0, 0, 0, 0, 0]

sol = minimize(f_num_for_minimize, x0, tol=1e-4)

(p_y_e, phi_e, theta_e, psi_e, v_x_e, v_y_e, v_z_e, 
 w_x_e, w_y_e, w_z_e, delta_r_e, delta_l_e) = sol.x

A = sym.lambdify(
    (p_y, psi, theta, phi, v_x, v_y, v_z, w_x, w_y, w_z, delta_r, delta_l), 
    f_mod.jacobian([p_y, psi, theta, phi, v_x, v_y, v_z, w_x, w_y, w_z]), 
    'numpy'
)

B = sym.lambdify(
    (p_y, psi, theta, phi, v_x, v_y, v_z, w_x, w_y, w_z, delta_r, delta_l), 
    f_mod.jacobian([delta_r, delta_l]), 
    'numpy'
)

A_num = A(p_y_e, psi_e, theta_e, phi_e, v_x_e, v_y_e, v_z_e, 
          w_x_e, w_y_e, w_z_e, delta_r_e, delta_l_e)

B_num = B(p_y_e, psi_e, theta_e, phi_e, v_x_e, v_y_e, v_z_e, 
          w_x_e, w_y_e, w_z_e, delta_r_e, delta_l_e)

A_num = np.array(A_num).astype(np.float64)
B_num = np.array(B_num).astype(np.float64)

# Solve the Continuous Algebraic Riccati Equation and compute the LQR gain matrix K
#     Args:
#         A (ndarray): State transition matrix
#         B (ndarray): Control input matrix
#         Q (ndarray): State cost matrix
#         R (ndarray): Control cost matrix
#     Returns:
#         K (ndarray): LQR gain matrix
#         P (ndarray): Solution to the Riccati equation
def lqr(A, B, Q, R):
    P = solve_continuous_are(A, B, Q, R) 
    K = np.linalg.inv(R) @ B.T @ P
    return K, P

# Weight matrix for states
# Changed to help determine a successful gain matrix K to implement in controller
# Only last 3 states changed because they coorespond to the angular velocities
# ------------------------------------------------------------------------------------------------
Q = np.diag([1, 1, 1, 1, 1, 1, 1, 3, 3, 3])
# ------------------------------------------------------------------------------------------------
Q = np.array(Q, dtype=np.float64)

# Weight matrix for control inputs
R = np.diag([1, 1])
R = np.array(R, dtype=np.float64)

K, P = lqr(A_num, B_num, Q, R)
K = np.array(K, dtype=np.float64)

print("x_e = ", sol)
print("A = ", A_num)
print("B = ", B_num)
print("K = ", K)


x_e =    message: Optimization terminated successfully.
  success: True
   status: 0
      fun: 1.5157652669333123e-11
        x: [ 0.000e+00  1.029e-05  2.797e-01 -3.455e-07  4.741e+00
            -5.336e-05  2.031e+00 -1.213e-06  3.807e-07 -3.030e-07
            -7.509e-01 -7.509e-01]
      nit: 28
      jac: [ 0.000e+00  1.622e-07  1.399e-06 -8.996e-08 -1.704e-06
            -9.419e-07 -8.786e-06 -1.696e-06  7.451e-06  3.747e-07
            -2.994e-06  1.228e-06]
 hess_inv: [[ 1.000e+00  0.000e+00 ...  0.000e+00  0.000e+00]
            [ 0.000e+00  3.032e+00 ...  1.783e+00 -1.793e+00]
            ...
            [ 0.000e+00  1.783e+00 ...  1.237e+00 -9.654e-01]
            [ 0.000e+00 -1.793e+00 ... -9.654e-01  1.240e+00]]
     nfev: 468
     njev: 36
A =  [[ 0.00000000e+00  5.11708937e+00 -2.22104729e-07 -2.03053409e+00
  -3.32101891e-07  1.00000000e+00 -1.03818109e-05  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00 -9.05540980e-08  3.96066103e-